# FRQ 2
>  Overview of FRQ 2.  Relationship of FRQ 2 to Project Based learning.
- title: Classes
- toc: true
- badges: false
- image: /images/apcsa_units.png
- type: ap
- week: 15
- comments: true

## Classes Background
> The 2019 FRQ #2 describes a problem of a Step Tracker.  The task is to write a StepTracker class.   Included is the constructor and any required instance variables and
methods.
- Assumption would be a integer or long steps instance variable
- Constructor: StepTracker tr = new StepTracker(10000);
- Get Methods with algorithms: tr.activeDays(); tr.averageSteps(); 
- Set Methods: tr.addDailySteps(9000);

## StepTracker to PBL 
> The StepTracker would be a pretty conventional POJO in PBL. As a PBL concept, StepTracking is something that is done by an individual in relation to their personal fitness. What object or objects would you need for StepTracking...
- Person Object, in thinking about this object it seems contact information and age would be important to this application.
- Define a steps attribute that contains steps 

In [ ]:
/*
Person is a POJO, Plain Old Java Object.
 */
@Data
@NoArgsConstructor
@AllArgsConstructor
@Entity
public class Person {
    // automatic unique identifier for Person record
    @Id
    @GeneratedValue(strategy = GenerationType.AUTO)
    private Long id;

    // email, password, roles are key to login and authentication
    @NotEmpty
    @Size(min=5)
    @Column(unique=true)
    @Email
    private String email;

    @NotEmpty
    private String password;

    // @NonNull: Places this in @RequiredArgsConstructor
    @NonNull
    @Size(min = 2, max = 30, message = "Name (2 to 30 chars)")
    private String name;

    @DateTimeFormat(pattern = "yyyy-MM-dd")
    private Date dob;

    // Steps attribute
    private ArrayList<Integer> steps = new ArrayList<>();
    
    // Initializer used when setting database from an API
    public Person(String email, String password, String name, Date dob) {
        this.email = email;
        this.password = password;
        this.name = name;
        this.dob = dob;
    }

    // A custom getter to return age from dob calculation
    public int getAge() {
        if (this.dob != null) {
            LocalDate birthDay = this.dob.toInstant().atZone(ZoneId.systemDefault()).toLocalDate();
            return Period.between(birthDay, LocalDate.now()).getYears(); }
        return -1;
    }

    public void addSteps(int steps) {
        this.steps.add(steps);
    }

}

## Limitations
> Flaws in definition. Quickly in my analysis of this POJO definition, I started to determine there would be some flaws as the definition seems to simple.  For instance...
- How could application determine relationship of Day of Week, Month, Year that steps were taken
- How could application update steps after initial add

### Collection, List, ArrayList
> The usage of Collection, List, or ArrayList enables a POJO to reference other POJOs.  Often in Java references you will see these different names...
- A Collection is an interface that defines the highest-level of shared collection behavior, and extends Iterable which allows usage of ForEach loop.
- A List is an interface that defines the highest-level of shared List behavior. The type List has in its contract that the elements have an order that order will not change except through intended manipulation.  List allows get() the n'th element.  List defines add() to put new elements at the end. 
- ArrayList is an implementation of List.  ArrayList should not be used in an API. Its usage should only be used in backend implementation detail.

## Hacks
> Redefine the POJO to be multiple POJOs.   One POJO contains Person, the second POJO contains Day.  The Collection, List ArrayList in Person would change to be a List of Day(s).  [Read for Background](https://vladmihalcea.com/the-best-way-to-map-a-onetomany-association-with-jpa-and-hibernate/).  This is great territory to understand Classes and Lists at the same time.

```java
// 1 to Many Relationship
// -----------              -----------
// | Person  | -|O-----|-<- | Day     |  
// -----------              -----------

@Entity
public class Person {
    // Person data

    // relationship
    @OneToMany(
        cascade = CascadeType.ALL,
        orphanRemoval = true
    )
    private List<Day> days = new ArrayList<>();
}

@Entity
public class Day {
    // attributes
    private int year;
    private int day;
 
    // relationship
    @ManyToOne(fetch = FetchType.LAZY)
    private Person person;
 
}
```